# Conversation summarization project
### BART and T5

## 1/ Import package

In [4]:
import pandas as pd
import os 
os.getcwd()

'c:\\Users\\romai\\Bureau\\DATA_SCIENCE\\PROJET_perso\\NLP\\Text_Summarization\\Text_summarization\\Notebook'

## 2/ Collect data

In [6]:
# import data (training part) : 

df_train = pd.read_parquet(r"..\Data\conv_train.parquet", engine="pyarrow")
df_val = pd.read_parquet(r"..\Data\conv_val.parquet", engine="pyarrow")
df_train

,id,dialogue,summary
0,13818513,Amanda: I baked cookies. Do you want some?\nJ...,Amanda baked cookies and will bring Jerry some...
1,13728867,Olivia: Who are you voting for in this electio...,Olivia and Olivier are voting for liberals in ...
2,13681000,"Tim: Hi, what's up?\nKim: Bad mood tbh, I was ...",Kim may try the pomodoro technique recommended...
3,13730747,"Edward: Rachel, I think I'm in ove with Bella....",Edward thinks he is in love with Bella. Rachel...
4,13728094,Sam: hey overheard rick say something\nSam: i...,"Sam is confused, because he overheard Rick com..."
...,...,...,...
14727,13863028,Romeo: You are on my ‘People you may know’ lis...,Romeo is trying to get Greta to add him to her...
14728,13828570,Theresa: <file_photo>\nTheresa: <file_photo>\n...,Theresa is at work. She gets free food and fre...
14729,13819050,John: Every day some bad news. Japan will hunt...,Japan is going to hunt whales again. Island an...
14730,13828395,Jennifer: Dear Celia! How are you doing?\nJenn...,Celia couldn't make it to the afternoon with t...


In [15]:
# Analyse some examples :
def print_example(df, txt_id) : 
    print(f"dialogue : \n{df["dialogue"].loc[txt_id]}")
    print(" ")
    print(f"summary : \n{df["summary"].loc[txt_id]}")

print_example(df_train, 10)

dialogue : 
Lucas: Hey! How was your day?
Demi: Hey there! 
Demi: It was pretty fine, actually, thank you!
Demi: I just got promoted! :D
Lucas: Whoa! Great news!
Lucas: Congratulations!
Lucas: Such a success has to be celebrated.
Demi: I agree! :D
Demi: Tonight at Death & Co.?
Lucas: Sure!
Lucas: See you there at 10pm?
Demi: Yeah! See you there! :D
 
summary : 
Demi got promoted. She will celebrate that with Lucas at Death & Co at 10 pm.


## 3/ Split df_train and df_test (already done) 
### or load the test set

In [19]:
df_test = pd.read_parquet(r"..\Data\conv_test.parquet", engine="pyarrow")
df_test

,id,dialogue,summary
0,13862856,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,13729565,Eric: MACHINE!\nRob: That's so gr8!\nEric: I k...,Eric and Rob are going to watch a stand-up on ...
2,13680171,"Lenny: Babe, can you help me with something?\n...",Lenny can't decide which trousers to buy. Bob ...
3,13729438,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,13828600,"Ollie: Hi , are you in Warsaw\nJane: yes, just...",Jane is in Warsaw. Ollie and Jane has a party....
...,...,...,...
814,13611902-1,Alex: Were you able to attend Friday night's b...,Benjamin didn't come to see a basketball game ...
815,13820989,Jamilla: remember that the audition starts at ...,The audition starts at 7.30 P.M. in Antena 3.
816,13717193,"Marta: <file_gif>\nMarta: Sorry girls, I click...","Marta sent a file accidentally,"
817,13829115,Cora: Have you heard how much fuss British med...,There was a meet-and-greet with James Charles ...


## 4.1/ Data cleaning 1

In [ ]:
# Check and rename col name : #Not needed
#df_train.rename(columns={"A": "a", "B": "c"})
#df_val.rename(columns={"A": "a", "B": "c"})
#df_test.rename(columns={"A": "a", "B": "c"})


In [18]:
# Check type of col : #All good
print("train")
for i in df_train.columns :
    print(f'{i} : {df_train[i].dtypes}')

print(" ")
print("val")
for i in df_val.columns :
    print(f'{i} : {df_val[i].dtypes}')

print(" ")
print("test")
for i in df_test.columns :
    print(f'{i} : {df_test[i].dtypes}')

train
id : object
dialogue : object
summary : object
 
val
id : object
dialogue : object
summary : object
 
test
id : object
dialogue : object
summary : object


## 4.2/ Data cleaning 2

In [21]:
# Convert df (pandas df format) to Dataset (huggingface format) :
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train

Dataset({
    features: ['id', 'dialogue', 'summary'],
    num_rows: 14732
})

In [22]:
# Concat "dataset_train, dataset_val, dataset_test" into a DatasetDict : 
from datasets import DatasetDict
Datadict = DatasetDict()

Datadict['train'] = dataset_train
Datadict['val'] = dataset_val
Datadict['test'] = dataset_test

Datadict

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    val: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
})

# Next step

In [ ]:
# Basic treatment (drop duplicate, clean text, ...so on) : 
def basic_treatment(df :pd.DataFrame, useless_columns : list, drop_duplicate=True) -> pd.DataFrame:
    """Perform basic data treatment on a DataFrame.

        This function performs basic data treatment on a given DataFrame:
        1. Removes duplicate rows.
        2. Drops specified useless columns.

        Args:
            df (pd.DataFrame): The input DataFrame to be treated.
            useless_columns (set): A set of column names to be removed from the DataFrame.

        Returns:
            pd.DataFrame: A DataFrame with duplicate rows removed, specified columns dropped,
            and string values converted to lowercase.
    """
    
    # step 1: drop duplicate
    if drop_duplicate==True :
        df = df.drop_duplicates()
        df = df.reset_index(drop = True)

    # step 2 : drop useless col 
    df = df.drop(list(useless_columns), axis=1)

    # step 3 : lowercase caracter 
    #df = df.applymap(lambda s : s.lower() if (type(s) == str and pd.isna(s)==False) else s) #map or applymap

    # step 4 : drop white space 
    df = df.applymap(lambda s : s.strip() if (isinstance(s, str) and pd.isna(s)==False) else s) 

    # step 5 : drop multiple(double, triple) space 
    df = df.applymap(lambda s:s.replace("  ", " ") if (isinstance(s, str) and pd.isna(s)==False) else s) 

    # step 6 : replace " " by "_" 
    df = df.applymap(lambda s:s.replace(" ", "_") if (isinstance(s, str) and pd.isna(s)==False) else s) 

    # step 7 : remove accent 
    #df = df.applymap(lambda s: unidecode(s) if (isinstance(s, str) and pd.isna(s)==False) else s) 

    return df
